<a href="https://colab.research.google.com/github/yueannewang12/datafrom-quickstart-production/blob/main/Chat_with_Spanner_Graph_ipynb_(Corporate_Graph_Example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This colab walks through how to chat with your Spanner Graph using natural language and address some common practical issues along the way.



## Setup

### Authentication

Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

If you are using Colab to run this notebook, use the cell below and continue.

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Install dependencies

Secondly, let's install some dependencies. Notably, some google LangChain integration libraries.

In [ ]:
%pip install --upgrade --quiet langchain-google-spanner langchain-google-vertexai langchain-experimental json-repair

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


### Setup your environment variables

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "google.com:cloud-spanner-demo"    # @param {type:"string"}
INSTANCE_ID = "mtyin-demo"   # @param {type:"string"}
DATABASE_ID = "chat-with-spanner-graph-demo"   # @param {type:"string"}
GRAPH_ID = "CorpGraph"  # @param {type:"string"}
DEFAULT_LLM_MODEL_NAME = "gemini-2.0-flash-001"         # @param {type:"string"}
DEFAULT_EMBEDDING_MODEL_NAME = "text-embedding-004"         # @param {type:"string"}
VERBOSE = True    # @param {type:"boolean"}
DEFAULT_CHAT_HISTORY_TABLE_NAME = "ChatHistory"

# Set the project id
!gcloud config set project {PROJECT_ID}
%env GOOGLE_CLOUD_PROJECT={PROJECT_ID}

Updated property [core/project].
env: GOOGLE_CLOUD_PROJECT=google.com:cloud-spanner-demo


In [ ]:
from langchain_google_spanner import SpannerGraphStore
from langchain_google_spanner import SpannerGraphQAChain
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai.embeddings import VertexAIEmbeddings
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document
from langchain.globals import set_verbose as set_langchain_verbose
from google.cloud import spanner

def get_verbose():
  return VERBOSE

def set_verbose(verbose):
  set_langchain_verbose(verbose)
  QA_CHAIN.verbose = verbose
  VERBOSE = verbose
  return VERBOSE

LLM = ChatVertexAI(model=DEFAULT_LLM_MODEL_NAME)
EMBEDDINGS = VertexAIEmbeddings(model_name=DEFAULT_EMBEDDING_MODEL_NAME)
GRAPH_STORE = SpannerGraphStore(
  instance_id=INSTANCE_ID,
  database_id=DATABASE_ID,
  graph_name=GRAPH_ID,
)
QA_CHAIN = SpannerGraphQAChain.from_llm(
  LLM,
  graph=GRAPH_STORE,
  return_intermediate_steps=True,
  allow_dangerous_requests=True,
  max_gql_fix_retries=3,
)
set_verbose(VERBOSE)

def get_database():
  spanner_client = spanner.Client(project=PROJECT_ID)
  instance = spanner_client.instance(INSTANCE_ID)
  database = instance.database(DATABASE_ID)
  return database

def query(q, **kwargs):
  print('User: {}'.format(q))
  try:
    print('LLM : {}'.format(QA_CHAIN.invoke({'query': q})['result']))
  except Exception as e:
    print("LLM : I don't know the answer: ERROR={}".format(e))

### Populate sample data (This is only required once)

In [ ]:
ddl_statements = [
  '''
  CREATE TABLE IF NOT EXISTS
  Company (
    company_id INT64 NOT NULL,
    name STRING(255),
    business_description STRING(MAX),
    year_start INT64,
  )
  PRIMARY KEY (company_id)
  ''',
  '''
  CREATE TABLE IF NOT EXISTS
  Person (
    person_id INT64 NOT NULL,
    full_name STRING(255),
    age INT64,
    )
  PRIMARY KEY (person_id)
  ''',
  '''
  CREATE TABLE IF NOT EXISTS
  Employment (
    company_id INT64 NOT NULL,
    person_id INT64 NOT NULL,
    employee_main_role STRING(100),
    is_board_member BOOL,
    FOREIGN KEY(person_id) REFERENCES Person(person_id),
    )
  PRIMARY KEY (company_id, person_id), INTERLEAVE IN PARENT Company
  ''',
  '''
  CREATE OR REPLACE PROPERTY GRAPH CorpGraph
  NODE TABLES (
    Person PROPERTIES(person_id, full_name, age),
    Company PROPERTIES(company_id, name, business_description, year_start),
  )
  EDGE TABLES (
    Employment AS hasEmployee
      SOURCE KEY(company_id) REFERENCES Company
      DESTINATION KEY(person_id) REFERENCES Person
      PROPERTIES (company_id, person_id, employee_main_role, is_board_member)
  )
  '''
]

get_database().update_ddl(ddl_statements)
GRAPH_STORE.impl.insert_or_update('Company',
                                        ['company_id', 'name', 'business_description', 'year_start'],
                                        [[1, 'Alphabet, Inc.', '''Alphabet Inc. is an American multinational technology conglomerate holding company headquartered in Mountain View, California. Alphabet is the world's third-largest technology company by revenue, after Amazon and Apple, and one of the world's most valuable companies.[2][3] It was created through a restructuring of Google on October 2, 2015,[4] and became the parent holding company of Google and several former Google subsidiaries.[5][6][7] It is considered one of the Big Five American information technology companies, alongside Amazon, Apple, Meta (owner of Facebook), and Microsoft.''', 2015],
                                         [2, 'Blackstone, Inc.', '''Blackstone Inc. is an American alternative investment management company based in New York City. It was founded in 1985 as a mergers and acquisitions firm by Peter Peterson and Stephen Schwarzman, who had previously worked together at Lehman Brothers. Blackstone's private equity business has been one of the largest investors in leveraged buyouts in the last three decades, while its real estate business has actively acquired commercial real estate across the globe. Blackstone is also active in credit, infrastructure, hedge funds, secondaries, growth equity, and insurance solutions. As of May 2024, Blackstone has more than US$1 trillion in total assets under management, making it the world's largest alternative investment firm.''', 1985],
                                         [3, 'Corning, Inc.', 'Corning Incorporated is an American multinational technology company that specializes in specialty glass, ceramics, and related materials and technologies including advanced optics, primarily for industrial and scientific applications.', 1851],
                                         [4, 'International Flavors & Fragrances, Inc.', 'International Flavors & Fragrances Inc. (IFF) is an American corporation that creates products across taste, texture, scent, nutrition, enzymes, cultures, soy proteins, and probiotics categories, which it markets globally. It is headquartered in New York City and has creative, sales, and manufacturing facilities in 44 countries. The company is a member of the S&P 500 index.', 1833]])
GRAPH_STORE.impl.insert_or_update('Person',
                                        ['person_id', 'full_name', 'age'],
                                        [[1, 'Sergey Mikhaylovich Brin', 49],
                                         [2, 'Lawrence E. Page', 50],
                                         [3, 'Sundar Pichai', 50],
                                         [4, 'Roger W. Ferguson Jr.', 71],
                                         [5, 'Ruth M. Porat', 65],
                                         [6, 'Larry Fink', 72]])
GRAPH_STORE.impl.insert_or_update('Employment',
                                        ['company_id', 'person_id', 'employee_main_role', 'is_board_member'],
                                        [[1, 3, 'Chief Executive Officer & Director', True],
                                         [1, 5, 'President & Chief Investment Officer', False],
                                         [2, 5, 'Independent Director', True],
                                         [2, 6, 'Co-founder, chairman and CEO', True],
                                         [1, 2, 'Co-founder', True],
                                         [1, 1, 'Co-founder', True],
                                         [1, 4, 'Independent Director', True],
                                         [3, 4, 'Independent Director', True],
                                         [4, 4, 'Chairman', True]])

In [ ]:
%%spanner_graph --project {PROJECT_ID} --instance {INSTANCE_ID} --database {DATABASE_ID}

GRAPH CorpGraph
MATCH p = (:Company) -[:hasEmployee]-> (:Person)
RETURN SAFE_TO_JSON(p) AS p

## Start to chat now!

In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph

QUERY = "Who're the employees of Alphabet, Inc.?"    # @param {type:"string"}
query(QUERY)

User: Who're the employees of Alphabet, Inc.?


> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH CorpGraph
MATCH (c:Company {name: 'Alphabet, Inc.'})-[:hasEmployee]->(p:Person)
RETURN p.full_name AS employee_name;
Full Context:
[{'employee_name': 'Sergey Mikhaylovich Brin'}, {'employee_name': 'Lawrence E. Page'}, {'employee_name': 'Sundar Pichai'}, {'employee_name': 'Roger W. Ferguson Jr.'}, {'employee_name': 'Ruth M. Porat'}]

> Finished chain.
LLM : Sergey Mikhaylovich Brin, Lawrence E. Page, Sundar Pichai, Roger W. Ferguson Jr., and Ruth M. Porat are the employees of Alphabet, Inc.



### Practical issues when chatting with Spanner Graph

In practice, there are some common issues

1.   LLM can sometimes generate invalid GQL ([Graph Query Language](https://cloud.google.com/spanner/docs/graph/queries-overview)) queries;

      For example, LLM might not know some functions or query features that are available in the query language. LLM can also sometimes confuses the syntax between different query languages.

      GQL as a query language is also constantly evolving with new features, which means LLM doesn't always have the most up-to-date knowledge about GQL.

1.   LLM sometimes gives incorrect edge directions.

     For example, find employees of company A.     

      *   ```(:Company) -[:hasEmployee]-> (:Person)```
      *   ```(:Person) -[:hasEmployee]-> (:Company)```

      This is related to 1, but sometimes the schema can be
      inherently ambiguous.

1.   LLM does not know the canonical references in your graph;

      For example, while the following queries have the same answer.

      *   Who's the CEO of Alphabet?
      *   Who's the CEO of Google?

      However, your Spanner Graph might not be able to answer the first version
      because the canonical reference in your graph may be `Google` instead of `Alphabet`.


     


## Better NL2GQL with semantic-guided examples

You *may* find the LLM cannot figure out the right query for the following question.

>  YMMV due to different LLM, schema or data. If it works, great! The general argument still applies: LLM out-of-box can sometimes generate incorrect queries.

In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph

QUERY = "Return full names of persons who are NOT employees of Alphabet, Inc."    # @param {type:"string"}
query(QUERY)

User: Return full names of persons who are NOT employees of Alphabet, Inc.


> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH CorpGraph
MATCH (c:Company {name: 'Alphabet, Inc.'})-[e:hasEmployee]->(p:Person)
WITH COLLECT(p) AS employees
MATCH (all_persons:Person)
WHERE all_persons NOT IN UNNEST(employees)
RETURN all_persons.full_name AS non_employee_full_name;
Invalid generated gql:
GRAPH CorpGraph
MATCH (c:Company {name: 'Alphabet, Inc.'})-[e:hasEmployee]->(p:Person)
WITH COLLECT(p) AS employees
MATCH (all_persons:Person)
WHERE all_persons NOT IN UNNEST(employees)
RETURN all_persons.full_name AS non_employee_full_name;
Query error: 
400 Function not found: COLLECT [at 3:6]\nWITH COLLECT(p) AS employees\n     ^ [locale: "en-US"
message: "Function not found: COLLECT [at 3:6]\nWITH COLLECT(p) AS employees\n     ^"
]
Executing gql:
GRAPH CorpGraph
MATCH (p:Person)
WHERE NOT EXISTS ((:Company {name: 'Alphabet, Inc.'})-[:hasEmployee]->(p))
RETURN p.full_name AS non_employee_f

### Improving NL2GQL with semantically similar examples

One simple strategy to mitigate such issue is to provide semantic guided
examples.

In the following example,

1.   We add a Spanner-based vector store that stores user intent and GQL examples;
2.   We extend the QA chain to pick semantically close GQL examples based on the user intent as a context for GQL generation.


nl2gql_step_one_example_guided_nl2gql.svg

### Define vector-based example store and use it for prompting

In [ ]:
from typing import Any, Callable, List, Optional, Union, Tuple
from langchain_community.graphs.graph_store import GraphStore
from langchain_core.vectorstores import VectorStore
from langchain_core import runnables
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.embeddings import Embeddings
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
from langchain_core.example_selectors.base import BaseExampleSelector
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_google_spanner import (
    SpannerGraphQAChain,
    SpannerGraphStore,
    SpannerVectorStore,
    TableColumn,
)
from langchain_google_spanner.prompts import (
    DEFAULT_GQL_FIX_TEMPLATE_PART0,
    DEFAULT_GQL_FIX_TEMPLATE_PART2,
    DEFAULT_GQL_TEMPLATE_PART1,
)

DEFAULT_GQL_EXAMPLE_PREFIX = """
Below are a number of examples of questions and their corresponding GQL queries.
"""
DEFAULT_GQL_EXAMPLE_TEMPLATE = """
Question:
  {question}
GQL Query:
  {gql}
"""
DEFAULT_GQL_GENERATION_WITH_EXAMPLE_PREFIX = """
You are a Spanner Graph Graph Query Language (GQL) expert.
Create an Spanner Graph GQL query for the question using the schema.
""" + DEFAULT_GQL_EXAMPLE_PREFIX
DEFAULT_GQL_FIX_TEMPLATE_WITH_EXAMPLE_PREFIX = (
    DEFAULT_GQL_FIX_TEMPLATE_PART0 + DEFAULT_GQL_EXAMPLE_PREFIX
)

class SpannerGQLExampleVectorStore(SpannerVectorStore):

  @staticmethod
  def from_embedding_with_spanner_config(
      instance_id: str,
      database_id: str,
      table_name: str,
      content_column: str,
      embedding_service: Optional[Embeddings] = None,
  ):
    if embedding_service is None:
      embedding_service = EMBEDDINGS
    gql_example_vector_store = SpannerGQLExampleVectorStore(
        instance_id,
        database_id,
        table_name,
        embedding_service=embedding_service,
        id_column="example_id",
        content_column=content_column,
        metadata_json_column="example",
    )
    return gql_example_vector_store

  def add_example(self, question: str, gql: str, schema: str = ""):
    self.add_texts(
        texts=[question],
        metadatas=[{
            "question": question,
            "gql": gql.replace("{", "{{").replace("}", "}}"),
            "schema": schema,
        }],
    )

def get_qa_chain_with_example(
    graph: Optional[GraphStore] = None,
    llm: Optional[BaseLanguageModel] = None,
    example_store: Optional[VectorStore] = None,
    *,
    gql_example_k = 3,
    embedding_service: Optional[Embeddings] = None,
    gql_instruction_prompt: Optional[str] = None,
    **kwargs: Any,
):
  if llm is None:
    llm = LLM

  example_selector = None
  if example_store is not None:
    example_selector = MaxMarginalRelevanceExampleSelector(
        vectorstore=example_store,
        k=gql_example_k,
    )

  if not gql_instruction_prompt:
    gql_instruction_prompt = DEFAULT_GQL_TEMPLATE_PART1

  gql_prompt = (
      FewShotPromptTemplate(
          example_selector=example_selector,
          example_prompt=PromptTemplate.from_template(
              DEFAULT_GQL_EXAMPLE_TEMPLATE
          ),
          prefix=DEFAULT_GQL_GENERATION_WITH_EXAMPLE_PREFIX,
          suffix=gql_instruction_prompt,
          input_variables=["question", "schema"],
      )
      if example_selector is not None
      else None
  )
  gql_fix_prompt = (
      FewShotPromptTemplate(
          example_selector=example_selector,
          example_prompt=PromptTemplate.from_template(
              DEFAULT_GQL_EXAMPLE_TEMPLATE
          ),
          prefix=DEFAULT_GQL_FIX_TEMPLATE_WITH_EXAMPLE_PREFIX,
          suffix=DEFAULT_GQL_FIX_TEMPLATE_PART2,
          input_variables=[
              "question",
              "generated_gql",
              "err_msg",
              "schema",
          ],
      )
      if example_selector is not None
      else None
  )
  return SpannerGraphQAChain.from_llm(
      llm,
      graph=graph,
      gql_prompt=gql_prompt,
      gql_fix_prompt=gql_fix_prompt,
      max_gql_fix_retries=3,
      **kwargs,
  )

In [ ]:
SpannerGQLExampleVectorStore.init_vector_store_table(
      INSTANCE_ID,
      DATABASE_ID,
      table_name='GqlExamples',
      id_column="example_id",
      content_column="question",
      metadata_columns=[
          TableColumn(name="example", type="JSON"),
      ],
)

Waiting for operation to complete...


True

In [ ]:
VECTOR_STORE = SpannerGQLExampleVectorStore.from_embedding_with_spanner_config(
  INSTANCE_ID,
  DATABASE_ID,
  'GqlExamples',
  content_column="question",
)

QA_CHAIN = get_qa_chain_with_example(
    llm=LLM,
    graph=GRAPH_STORE,
    example_store=VECTOR_STORE,
    return_intermediate_steps=True,
    allow_dangerous_requests=True,
)

### Add example queries

> NOTE: A general rule of thumb is to use parameterized queries to avoid semantics diluted by specific entity names.

In [ ]:
EXAMPLE_NL_QUERY = "Return persons who are not affiliated with company `@name`"    # @param {type:"string"}
EXAMPLE_GQL_QUERY = """
GRAPH CorpGraph
MATCH (n:Person)
WHERE NOT EXISTS {
  MATCH (m:Person) <-[:hasEmployee]- (:Company {name: @name}) -- @name is a parameter
  WHERE m = n   -- use an explicit filter instead of redeclaring `n`
}
RETURN n.full_name
"""
VECTOR_STORE.add_example(EXAMPLE_NL_QUERY, EXAMPLE_GQL_QUERY)

In [ ]:
EXAMPLE_NL_QUERY = "How many employees for each company in `@company_names`"    # @param {type:"string"}
EXAMPLE_GQL_QUERY = """
GRAPH CorpGraph
MATCH (n:Company) -[:hasEmployee]-> (:Person)
WHERE n.name IN UNNEST(@company_names) -- @company_names is a parameter
RETURN n.name, COUNT(*) AS cnt
"""
VECTOR_STORE.add_example(EXAMPLE_NL_QUERY, EXAMPLE_GQL_QUERY)

In [ ]:
EXAMPLE_NL_QUERY = "Which companies have more than @k employees?"    # @param {type:"string"}
EXAMPLE_GQL_QUERY = """
GRAPH CorpGraph
MATCH (n:Company) -[:hasEmployee]-> (:Person)
WITH n.name, COUNT(*) AS cnt
FILTER cnt > @k -- @k is a parameter
RETURN n.name, cnt
"""
VECTOR_STORE.add_example(EXAMPLE_NL_QUERY, EXAMPLE_GQL_QUERY)

In [ ]:
EXAMPLE_NL_QUERY = "Find persons that are employed by both @companyNameA and @companyNameB?"    # @param {type:"string"}
EXAMPLE_GQL_QUERY = """
GRAPH CorpGraph
MATCH (n:Company {name: @companyNameA}) -[:hasEmployee]-> (p:Person),
      (m:Company {name: @companyNameB}) -[:hasEmployee]-> (p:Person)
RETURN p.full_name
"""
VECTOR_STORE.add_example(EXAMPLE_NL_QUERY, EXAMPLE_GQL_QUERY)

### Chat with GQL examples

In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Return full names of persons who are NOT employees of Alphabet, Inc."    # @param {type:"string"}
examples = MaxMarginalRelevanceExampleSelector(vectorstore=VECTOR_STORE, k=3).select_examples({'question': QUERY})
print('Selecting GQL examples:')
for example in examples:
  print("===")
  print(example['question'], example['gql'].format())

print('Answering user questions:')
query(QUERY)

Selecting GQL examples:
===
Return persons who are not affiliated with company `@name` 
GRAPH CorpGraph
MATCH (n:Person)
WHERE NOT EXISTS {
  MATCH (m:Person) <-[:hasEmployee]- (:Company {name: @name}) -- @name is a parameter
  WHERE m = n   -- use an explicit filter instead of redeclaring `n`
}
RETURN n.full_name

===
Which companies have more than @k employees? 
GRAPH CorpGraph
MATCH (n:Company) -[:hasEmployee]-> (:Person)
WITH n.name, COUNT(*) AS cnt
FILTER cnt > @k -- @k is a parameter
RETURN n.name, cnt

===
How many employees for each company in `@company_names` 
GRAPH CorpGraph
MATCH (n:Company) -[:hasEmployee]-> (:Person)
WHERE n.name IN UNNEST(@company_names) -- @company_names is a parameter
RETURN n.name, COUNT(*) AS cnt

Answering user questions:
User: Return full names of persons who are NOT employees of Alphabet, Inc.


> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH CorpGraph
MATCH (n:Person)
WHERE NOT EXISTS {
  MATCH (:Company {name: "Alphabet, Inc."}) 

## (WIP) Better NL2GQL with domain knowledge guided prompts



In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph

QUERY = "Find persons who are affiliated to Alphabet, Inc."    # @param {type:"string"}
query(QUERY)

User: Find persons who are affiliated to Alphabet, Inc.
LLM : Sergey Mikhaylovich Brin, Lawrence E. Page, Sundar Pichai, Roger W. Ferguson Jr., and Ruth M. Porat are affiliated to Alphabet, Inc.



In [ ]:
GQL_INSTRUCTION_PROMPT = """
Instructions:

  * The allowed one hop patterns are
      [
        (company:Company)-[:hasEmployee]->(employee:Person),
      ].

    You must choose patterns from the list or composite patterns from the list
    to form more complex patterns.
"""

QA_CHAIN_WIP = get_qa_chain_with_example(
    llm=LLM,
    graph=GRAPH_STORE,
    example_store=VECTOR_STORE,
    return_intermediate_steps=True,
    allow_dangerous_requests=True,
)

## Query rewrite

Sometimes even if the GQL syntax is correct, you still won't get the correct results due to uncanonical references.

In the following example, even though the query syntax is correct, the answer is incorrect because the canonical reference is `Alphabet Inc.` instead of `Alphabet`.

This section illustrates how to use Spanner's built-in capabilities such as vector-search and full-text search to improve the QA quality.

In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Who're employees of Alphabet?"    # @param {type:"string"}
query(QUERY)

User: Who're employees of Alphabet?
LLM : I don't know the answer.


### Add search index (This is only required once)

In [ ]:
operation = get_database().update_ddl([
  """
  ALTER TABLE Company
  ADD COLUMN IF NOT EXISTS name_token TOKENLIST AS
    (TOKENIZE_SUBSTRING(name, ngram_size_min=>3, ngram_size_max=>4)) HIDDEN
  """,
  """
  DROP SEARCH INDEX IF EXISTS CompanyNameSearchIndex
  """,
  """
  CREATE SEARCH INDEX CompanyNameSearchIndex ON Company(name_token) STORING (name)
  """,
  """
  ALTER TABLE Person
  ADD COLUMN IF NOT EXISTS full_name_token TOKENLIST AS
    (TOKENIZE_SUBSTRING(full_name, ngram_size_min=>3, ngram_size_max=>4)) HIDDEN
  """,
  """
  DROP SEARCH INDEX IF EXISTS PersonNameSearchIndex
  """,
  """
  CREATE SEARCH INDEX PersonNameSearchIndex ON Person(full_name_token) STORING (full_name)
  """,
])
operation.result()

### Query rewrite with full-text search

In [ ]:
from langchain.tools import Tool
from pydantic import BaseModel, Field

def rewrite_question_by_references(question: str, references: List[str], query: str) -> Tuple[str, List[dict[str, str]]]:

  try:
    spanner_client = spanner.Client(project=PROJECT_ID)
    instance = spanner_client.instance(INSTANCE_ID)
    database = instance.database(DATABASE_ID)
    results = []
    for reference in references:
      with get_database().snapshot() as snapshot:
        rows = snapshot.execute_sql(
          query,
          params={"reference": reference},
          param_types={'reference': spanner.param_types.STRING},
        )
        values = [row[0] for row in rows]
        if values:
          results.append({'reference': reference, 'canonical_reference': values[0]})
  except Exception as e:
    print("Failed to find relevant entities: %s" % e)
    results = []

  for result in results:
    reference, canonical_reference = result['reference'], result['canonical_reference']
    if get_verbose():
      print(f'>> Rewriting `{reference}` into `{canonical_reference}`...')
    question = question.replace(reference, canonical_reference)
  return question, results

doc = """Rewrites the user question which contains {category} references.

     `{category}_references`:
        The {category} references in the question. They can be
        - full names
        - partial names (e.g. first name, last name, full name without middle name)
        - nick names or aliases.

        Do not use when the references are pronounces.
        There can be multiple {category} references in a question.

      Returns:
        Tuple of (the rewritten question, a list of mapping from the reference to canonicali reference)

        When a user asks about {category} with `{category}_references`,
        rewrite_question_by_{category}_references({category}_references)
        rewrites `question` with a more accurate reference.

    For example,

      rewrite_question_by_{category}_references('What's the id of `{example_reference}`?', ['{example_reference}'])
      may rewrite the query into "What's the id of `{example_canonical_reference}`?" and returns
      [{{{{'reference': '{example_reference}', 'canonical_reference': '{example_canonical_reference}'}}}}].

    This function is usually called before `chat_with_spanner_graph` to rewrite
    the user question.
  """

def rewrite_question_by_person_references(question: str, person_references: List[str]) -> Tuple[str, List[dict[str, str]]]:
  return rewrite_question_by_references(
          question,
          person_references,
          """
            SELECT full_name AS canonical_reference
            FROM Person
            WHERE SEARCH_SUBSTRING(full_name_token, @reference) AND
                  SCORE_NGRAMS(full_name_token, @reference) > 0.2
            ORDER BY SCORE_NGRAMS(full_name_token, @reference) DESC
            LIMIT 1
          """)

def rewrite_question_by_company_references(question: str, company_references: List[str]) -> Tuple[str, List[dict[str, str]]]:
  return rewrite_question_by_references(
          question,
          company_references,
          """
            SELECT name AS canonical_reference
            FROM Company
            WHERE SEARCH_SUBSTRING(name_token, @reference) AND
                  SCORE_NGRAMS(name_token, @reference) > 0.2
            ORDER BY SCORE_NGRAMS(name_token, @reference) DESC
            LIMIT 1
          """)

rewrite_question_by_person_references.__doc__ = doc.format(category='person', example_reference='Sundar Pichai', example_canonical_reference='Pichai Sundararajan')
rewrite_question_by_company_references.__doc__ = doc.format(category='company', example_reference='Google', example_canonical_reference='Google LLC')

def chat_with_spanner_graph(
    maybe_rewritten_question: str
) -> str:
  """Answers user question by talking to Spanner Graph.

  `maybe_rewritten_question` can be a new question rewritten by other tools
  when there are references in the original question. Otherwise, it will be
  the original user query.

  Returns:
    The answer from Spanner Graph.
  """

  if get_verbose():
    print(f'>> Answering question: `{maybe_rewritten_question}`...')
  results = QA_CHAIN.invoke({"query": maybe_rewritten_question})
  return results['result']

In [ ]:
rewritten_question, mapping = rewrite_question_by_company_references('Which year is Alphabet founded?', ['Alphabet'])
print(chat_with_spanner_graph(rewritten_question))

Alphabet, Inc. was founded in 2015.



### Query rewrite with agents

Agents enable automatical orchestration of tools using the LLM.

In [ ]:
from vertexai.preview.reasoning_engines import LangchainAgent

tools = [rewrite_question_by_person_references, rewrite_question_by_company_references, chat_with_spanner_graph]
system_prompt = """
You're a helpful assistant that uses the given tools to answer questions.

Instructions:
1) Look at the available tools and figure out which ones can be relevant;
   Note that: multiple tools could be used;
2) Examine possible ways of orchestrating the tools:
   Note that: consider different orders the tools could be executed;
              same tools could be executed multiple times;
              the output of preceding rules could be used as a context for the following rule;
              the last tool should answer the user question.
3) Pick the best plan and excute it.
4) Return "I don't know" if you still cannot get the right answer.
"""

# def get_tool_prompt(tool):
#   return "===\nTool name: {},\nTool documentation: {}\n\n".format(tool.__name__, tool.__doc__)

# system_prompt += """
# Tools:

# {}
# """.format('\n'.join(map(get_tool_prompt, tools)))
print(system_prompt)

AGENT = LangchainAgent(
    model=DEFAULT_LLM_MODEL_NAME,
    system_instruction=system_prompt,
    tools=tools,
)

def query(q, **kwargs):
  print('User: {}'.format(q))
  try:
    print('LLM : {}'.format(AGENT.query(input=q, **kwargs)['output']))
  except Exception as e:
    print("LLM : I don't know the answer: ERROR={}".format(e))


You're a helpful assistant that uses the given tools to answer questions.

Instructions:
1) Look at the available tools and figure out which ones can be relevant;
   Note that: multiple tools could be used;
2) Examine possible ways of orchestrating the tools:
   Note that: consider different orders the tools could be executed;
              same tools could be executed multiple times;
              the output of preceding rules could be used as a context for the following rule;
              the last tool should answer the user question.
3) Pick the best plan and excute it.
4) Return "I don't know" if you still cannot get the right answer.



In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Which year was Alphabet founded?"    # @param {type:"string"}
query(QUERY)

User: Which year was Alphabet founded?
LLM : Alphabet, Inc. was founded in 2015.



In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "What're Ruth Porat's roles in the companies she is affiliated to?"    # @param {type:"string"}
query(QUERY)

User: What're Ruth Porat's roles in the companies she is affiliated to?
LLM : Ruth M. Porat's roles are President & Chief Investment Officer and Independent Director.



In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "What're Ruth Porat's roles in Alphabet?"    # @param {type:"string"}
query(QUERY)

User: What're Ruth Porat's roles in Alphabet?
LLM : I don't know the answer.


## Make it conversational

In real-world applications, it's common to have follow up questions in a conversation.

For example, in the following example, our agent cannot answer the follow up
question because it doesn't have conversation history to figure out what does
`her` refer to in the question.

In [ ]:
# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Which companies does Ruth Porat work at?"    # @param {type:"string"}
query(QUERY)

print()

# @markdown Type your query here and run the call to chat with Spanner Graph
FOLLOWUP_QUERY = "What are her roles in these companies?"    # @param {type:"string"}
query(FOLLOWUP_QUERY)

User: Which companies does Ruth Porat work at?
LLM : Ruth M. Porat works at Alphabet, Inc. and Blackstone, Inc.


User: What are her roles in these companies?
LLM : Could you please provide the name of the person and the companies you are asking about?



### Add chat history to agent

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_google_spanner.chat_message_history import SpannerChatMessageHistory

SpannerChatMessageHistory.create_chat_history_table(
    instance_id=INSTANCE_ID,
    database_id=DATABASE_ID,
    table_name=DEFAULT_CHAT_HISTORY_TABLE_NAME,
)

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_google_spanner.chat_message_history import SpannerChatMessageHistory

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
  return SpannerChatMessageHistory(
        instance_id=INSTANCE_ID,
        database_id=DATABASE_ID,
        table_name=DEFAULT_CHAT_HISTORY_TABLE_NAME,
        session_id=session_id,
    )

AGENT = LangchainAgent(
    model=DEFAULT_LLM_MODEL_NAME,
    system_instruction=system_prompt,
    tools=tools,
    chat_history=get_by_session_id,
)

### Chat with history

If you lucky, the following query and its followup now should work.

In [ ]:
import uuid
from pprint import pprint
session_id = str(uuid.uuid4())
print('Session id:', session_id)

# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Which companies does Ruth Porat work at?"    # @param {type:"string"}
query(QUERY, config={"configurable": {"session_id": session_id}})

print()

# @markdown Type your query here and run the call to chat with Spanner Graph
FOLLOWUP_QUERY = "What are her roles in these companies?"    # @param {type:"string"}
query(FOLLOWUP_QUERY, config={"configurable": {"session_id": session_id}})

Session id: 53dab66c-84d8-4ed7-bc9b-0dd8e6469c7d
User: Which companies does Ruth Porat work at?


> Entering new AgentExecutor chain...

Invoking: `rewrite_question_by_person_references` with `{'person_references': ['Ruth Porat'], 'question': 'Which companies does Ruth Porat work at?'}`
responded: I should use the tool `rewrite_question_by_person_references` to rewrite the question with a more accurate reference, then use the tool `chat_with_spanner_graph` to answer the question.


>> Rewriting `Ruth Porat` into `Ruth M. Porat`...
('Which companies does Ruth M. Porat work at?', [{'reference': 'Ruth Porat', 'canonical_reference': 'Ruth M. Porat'}])
Invoking: `chat_with_spanner_graph` with `{'maybe_rewritten_question': 'Which companies does Ruth M. Porat work at?'}`


>> Answering question: `Which companies does Ruth M. Porat work at?`...


> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH CorpGraph
MATCH (c:Company) -[:hasEmployee]-> (p:Person {full_name: 'Ruth M. Porat'}

### Tune the tool interface design for better tool orchestration

But in reality, it gets tricker than that.

Sometimes, LLM is able to figure out `her` means `Ruth Porat` but failed to
understand we should use `Ruth M. Porat` even though we might already resolve
the reference before. And sometimes, LLM decided not to invoke the reference resolution tool the second time.

The following example shows another possible way to design the tool interface.
Again, YMMV depends on the exact LLM, schema, query and tools.

In [ ]:
def chat_with_spanner_graph(
    question: str,
    person_references: List[str] = [],
    company_references: List[str] = [],
) -> str:
  """Answers user question by talking to Spanner Graph.

  `question`:
      user input question;

  `person_references`:
      A list of person references in the question;
      It can be empty when there is no person reference in the question.

  `company_references`:
      A list of company references in the question;
      It can be empty when there is company person reference in the question.

  Returns:
    The answer from Spanner Graph.
  """
  if get_verbose():
    print(f'>> Receiving question: `{question}`...')
  canonical_person_references = []
  canonical_company_references = []

  if person_references:
    if get_verbose():
      print(f'>> Receiving person_references: `{person_references}`...')
    question, canonical_person_references = rewrite_question_by_person_references(question, person_references)
    canonical_person_references = [ref['canonical_reference'] for ref in canonical_person_references]

  if company_references:
    if get_verbose():
      print(f'>> Receiving company_references: `{company_references}`...')
    question, canonical_company_references = rewrite_question_by_company_references(question, company_references)
    canonical_company_references = [ref['canonical_reference'] for ref in canonical_company_references]

  question = f"""
  {question}

  Canonical person references:
  {canonical_person_references}

  Canonical company references:
  {canonical_company_references}
  """

  if get_verbose():
    print(f'>> Answering question: `{question}`...')
  results = QA_CHAIN.invoke({"query": question})
  return results['result']

In [ ]:
tools = [chat_with_spanner_graph]
system_prompt = """
You're a helpful assistant that uses the given tools to answer questions.

Instructions:
1) Look at the available tools and figure out which ones can be relevant;
   Note that: multiple tools could be used;
2) Examine possible ways of orchestrating the tools:
   Note that: consider different orders the tools could be executed;
              same tools could be executed multiple times;
              the output of preceding rules could be used as a context for the following rule;
              the last tool should answer the user question.
3) Pick the best plan and excute it.
4) Return "I don't know" if you still cannot get the right answer.
"""

# def get_tool_prompt(tool):
#   return "===\nTool name: {},\nTool documentation: {}\n\n".format(tool.__name__, tool.__doc__)

# system_prompt += """
# Tools:

# {}
# """.format('\n'.join(map(get_tool_prompt, tools)))
print(system_prompt)

AGENT = LangchainAgent(
    model=DEFAULT_LLM_MODEL_NAME,
    system_instruction=system_prompt,
    tools=tools,
    chat_history=get_by_session_id,
)


You're a helpful assistant that uses the given tools to answer questions.

Instructions:
1) Look at the available tools and figure out which ones can be relevant;
   Note that: multiple tools could be used;
2) Examine possible ways of orchestrating the tools:
   Note that: consider different orders the tools could be executed;
              same tools could be executed multiple times;
              the output of preceding rules could be used as a context for the following rule;
              the last tool should answer the user question.
3) Pick the best plan and excute it.
4) Return "I don't know" if you still cannot get the right answer.



In [ ]:
print('Session id:', session_id)

# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Which companies does Ruth Porat work at?"    # @param {type:"string"}
query(QUERY, config={"configurable": {"session_id": session_id}})

print()

# @markdown Type your query here and run the call to chat with Spanner Graph
FOLLOWUP_QUERY = "What are her roles in these companies?"    # @param {type:"string"}
query(FOLLOWUP_QUERY, config={"configurable": {"session_id": session_id}})

Session id: c9ea9037-cec8-413c-80ec-5c75ff06d1dd
User: Which companies does Ruth Porat work at?
LLM : Alphabet, Inc. and Blackstone, Inc. are companies where Ruth M. Porat works.


User: What are her roles in these companies?
LLM : Ruth M. Porat's roles are President & Chief Investment Officer at Alphabet, Inc. and Independent Director at Blackstone, Inc.



### Show the chat session history

In [ ]:
for msg in get_by_session_id(session_id).messages:
  print(msg)

content='Which companies does Ruth Porat work at?' additional_kwargs={} response_metadata={}
content='Alphabet, Inc. and Blackstone, Inc. are the companies that Ruth M. Porat works at.' additional_kwargs={} response_metadata={}
content='What are her roles in these companies?' additional_kwargs={} response_metadata={}
content="I don't know the answer." additional_kwargs={} response_metadata={}
content='Which companies does Ruth Porat work at?' additional_kwargs={} response_metadata={}
content='Alphabet, Inc. and Blackstone, Inc. are companies where Ruth M. Porat works.\n' additional_kwargs={} response_metadata={}
content='What are her roles in these companies?' additional_kwargs={} response_metadata={}
content="Ruth M. Porat's roles are President & Chief Investment Officer at Alphabet, Inc. and Independent Director at Blackstone, Inc.\n" additional_kwargs={} response_metadata={}


## Playground

In [ ]:
set_langchain_verbose(True)
QA_CHAIN.verbose = True

import uuid
from pprint import pprint
session_id = str(uuid.uuid4())
print('Session id:', session_id)

# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Who're employed by more than one companies?"    # @param {type:"string"}
query(QUERY, config={"configurable": {"session_id": session_id}})

print()

# @markdown Type your query here and run the call to chat with Spanner Graph
FOLLOWUP_QUERY = "What the roles of these persons in those companies?"    # @param {type:"string"}
query(FOLLOWUP_QUERY, config={"configurable": {"session_id": session_id}})

Session id: c24d62d1-e11d-4621-969a-c8009d821082
User: Who're employed by more than one companies?


> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH CorpGraph
MATCH (c:Company) -[:hasEmployee]-> (p:Person)
WITH p, COUNT(c) AS num_companies
FILTER num_companies > 1
RETURN p.full_name AS employee_name
Full Context:
[{'employee_name': 'Roger W. Ferguson Jr.'}, {'employee_name': 'Ruth M. Porat'}]

> Finished chain.
LLM : Roger W. Ferguson Jr. and Ruth M. Porat are employed by more than one companies.


User: What the roles of these persons in those companies?
LLM : I am sorry, I cannot fulfill this request. The available tools do not allow me to query the roles of people in companies.



In [ ]:
query("Why don't you use the chat_with_spanner_graph tool?", config={"configurable": {"session_id": session_id}})

User: Why don't you use the chat_with_spanner_graph tool?


> Entering new SpannerGraphQAChain chain...
Executing gql:
GRAPH CorpGraph
MATCH (c:Company) -[e:hasEmployee]-> (p:Person)
WHERE p.full_name IN ("Roger W. Ferguson Jr.", "Ruth M. Porat")
RETURN p.full_name AS person_name, c.name AS company_name, e.employee_main_role AS role
Full Context:
[{'person_name': 'Roger W. Ferguson Jr.', 'company_name': 'Alphabet, Inc.', 'role': 'Independent Director'}, {'person_name': 'Roger W. Ferguson Jr.', 'company_name': 'Corning, Inc.', 'role': 'Independent Director'}, {'person_name': 'Roger W. Ferguson Jr.', 'company_name': 'International Flavors & Fragrances, Inc.', 'role': 'Chairman'}, {'person_name': 'Ruth M. Porat', 'company_name': 'Alphabet, Inc.', 'role': 'President & Chief Investment Officer'}, {'person_name': 'Ruth M. Porat', 'company_name': 'Blackstone, Inc.', 'role': 'Independent Director'}]

> Finished chain.
LLM : Roger W. Ferguson Jr. is the Independent Director at Alphabet, Inc. an

In [ ]:
import uuid
from pprint import pprint
session_id = str(uuid.uuid4())
print('Session id:', session_id)

# @markdown Type your query here and run the call to chat with Spanner Graph
QUERY = "Does Alphabet and Blackstone have any common employee?"    # @param {type:"string"}
query(QUERY, config={"configurable": {"session_id": session_id}})

# @markdown Type your query here and run the call to chat with Spanner Graph
FOLLOWUP_QUERY = "What are the names of these employees?"    # @param {type:"string"}
query(FOLLOWUP_QUERY, config={"configurable": {"session_id": session_id}})

# @markdown Type your query here and run the call to chat with Spanner Graph
FOLLOWUP_QUERY2 = "What are their roles in these companies?"    # @param {type:"string"}
query(FOLLOWUP_QUERY2, config={"configurable": {"session_id": session_id}})



Session id: 79623d49-0b42-40a7-9b77-394b1834b4ab
User: Does Alphabet and Blackstone have any common employee?
LLM : Yes, Alphabet and Blackstone have a common employee named Ruth M. Porat.

User: What are the names of these employees?
LLM : The name of the common employee between Alphabet and Blackstone is Ruth M. Porat.

User: What are their roles in these companies?
LLM : I am sorry, I cannot fulfill this request. The available tool does not provide information about the roles of the employees in these companies.

